# A CNN music genre classification model using mel-spectrogram images from 30 sec audio file.

source data: smMELsg_30sec_Train_Test.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ln -sf ./drive/MyDrive/projects/FNN_Project/data /content/data
!ln -sf ./drive/MyDrive/projects/FNN_Project/ref /content/ref
!cp ./ref/fma/utils.py ./
!pip install python-dotenv
!pip install scikeras
!pip install pydub

## Check image dimensions.

In [ ]:
%%time
import cv2
import os

content_mel_dir = '/content/fma_small_specgram_img'

for genre in ['Pop', 'Rock', 'Electronic']:
  img_shapes = {}
  for filename in os.listdir(content_mel_dir + '/train/' + f"{genre}"):
    track_path = f'{content_mel_dir}/train/{genre}/{filename}'
    im = cv2.imread(track_path)
    if im.shape not in img_shapes:
      img_shapes[im.shape] = 1
    else:
      img_shapes[im.shape] += 1
  print(img_shapes)

{(450, 610, 3): 800}
{(450, 610, 3): 799}
{(450, 610, 3): 799}
CPU times: user 25.3 s, sys: 222 ms, total: 25.5 s
Wall time: 25.4 s


## Define CNN model.

In [ ]:
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten,
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,
                          Dropout)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.optimizers import Adam

def GenreModel(input_shape = (450,610,3),classes=3):

  X_input = Input(input_shape)

  X = Conv2D(8,kernel_size=(3,3),strides=(1,1))(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(16,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(32,kernel_size=(3,3),strides = (1,1))(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(64,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Conv2D(128,kernel_size=(3,3),strides=(1,1))(X)
  X = BatchNormalization(axis=-1)(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((2,2))(X)

  X = Flatten()(X)

  X = Dropout(rate=0.3)(X)

  X = Dense(classes, activation='softmax', name='fc' + str(classes))(X)

  model = Model(inputs=X_input,outputs=X,name='GenreModel')

  return model

In [ ]:
#model = keras.models.load_model('./CNN_Split_v1.keras')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def get_data_generator(_source_dir, _shuffle):
  datagen = ImageDataGenerator(rescale=1./255)
  train_datagen = ImageDataGenerator(rescale=1./255)
  train_generator = train_datagen.flow_from_directory(_source_dir,target_size=(450,610),
                                                    color_mode="rgb",
                                                    class_mode='categorical',
                                                    batch_size=128,
                                                    shuffle = _shuffle)
  return train_generator

In [ ]:
train_generator = get_data_generator(content_mel_dir + '/train', True)
test_generator = get_data_generator( content_mel_dir +'/test', False)

Found 2398 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


## Fit model using all training data and test data as validation.

In [ ]:
%%time
model = GenreModel()
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=test_generator, epochs=30)
model.save('/content/data/mel_sg/models/CNN_30sec_img_RPE_m1_epoch30.keras')
!cp /content/data/mel_sg/models/CNN_30sec_img_RPE_m1_epoch30.keras ./data/mel_sg/

<timed exec>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
19/19 [==============================] - 49s 2s/step - loss: 2.7873 - accuracy: 0.4616 - val_loss: 1.1262 - val_accuracy: 0.4867
Epoch 2/30
19/19 [==============================] - 46s 2s/step - loss: 1.1261 - accuracy: 0.5459 - val_loss: 1.2945 - val_accuracy: 0.3333
Epoch 3/30
19/19 [==============================] - 46s 2s/step - loss: 0.9236 - accuracy: 0.5959 - val_loss: 1.5062 - val_accuracy: 0.3333
Epoch 4/30
19/19 [==============================] - 46s 2s/step - loss: 0.8836 - accuracy: 0.6230 - val_loss: 2.0392 - val_accuracy: 0.3333
Epoch 5/30
19/19 [==============================] - 46s 2s/step - loss: 0.8350 - accuracy: 0.6489 - val_loss: 1.9923 - val_accuracy: 0.3333
Epoch 6/30
19/19 [==============================] - 46s 2s/step - loss: 0.7192 - accuracy: 0.6897 - val_loss: 1.5714 - val_accuracy: 0.3333
Epoch 7/30
19/19 [==============================] - 46s 2s/step - loss: 0.7056 - accuracy: 0.7093 - val_loss: 1.9493 - val_accuracy: 0.3333
Epoch 8/30
19/19 [==

In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('/content/data/mel_sg/models/CNN_30sec_img_RPE_m1_epoch30.keras')
loaded_model.summary()

Model: "GenreModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 450, 610, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 448, 608, 8)       224       
                                                                 
 batch_normalization_15 (Ba  (None, 448, 608, 8)       32        
 tchNormalization)                                               
                                                                 
 activation_15 (Activation)  (None, 448, 608, 8)       0         
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 224, 304, 8)       0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 222, 302, 16)      1

# Perform 5-fold Cross validation

In [ ]:
%%time
import os

def get_file_paths(_split):
  file_paths = []
  for genre in ['Pop', 'Rock', 'Electronic']:
    img_shapes = {}
    for filename in os.listdir(content_mel_dir + '/' + _split + '/' + f"{genre}"):
      track_path = f'{content_mel_dir}/{_split}/{genre}/{filename}'
      file_paths.append((track_path, genre, filename))
  len(file_paths)
  return file_paths

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 12.2 µs


In [ ]:
train_file_paths = get_file_paths('train')
train_file_paths[0]

('/content/fma_small_specgram_img/train/Pop/48457.png', 'Pop', '48457.png')

## Shuffle paths to image files.

In [ ]:
%%time
import random

random.seed( 0 )

random.shuffle(train_file_paths)

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

n_folds = 5
k_folds = list(split(train_file_paths, n_folds))
print(len(train_file_paths))
print()
for f in k_folds:
  print(len(f))
print()
print(sum([len(f) for f in k_folds]))
print(len(k_folds))

2398

480
480
480
479
479

2398
5
CPU times: user 616 µs, sys: 797 µs, total: 1.41 ms
Wall time: 1.38 ms


## Create 5 folds of image files by copying source images into fold directories.

In [ ]:
%%time

#BE SURE TO DELTE OLD /content/kfolds directory.

import os
import shutil
import itertools

def copy_fold(_fold, _target_dir):
  for f in _fold:
    src = f[0]
    genre = f[1]
    filename = f[2]

    genre_dir = f'{_target_dir}/{genre}'
    dst = f'{genre_dir}/{filename}'
    os.makedirs(genre_dir, exist_ok=True)
    shutil.copyfile(src, dst)

n_folds = len(k_folds)
#Create combinations of folds, excluding one fold for test.
fold_combos = list(itertools.combinations(list(range(n_folds)), n_folds-1))
#Get test folds
test_folds = [idx for fc in fold_combos for idx in range(n_folds) if idx not in fc]
print(f'Train folds: {fold_combos}')
print(f'Test folds: {test_folds}')

for fold_idx, fc in enumerate(fold_combos):
  print(f'Fold: {fold_idx}')
  fold_dir = '/content/kfolds/fold' + str(fold_idx)
  train_target_dir = fold_dir +'/train'
  print(f'  Train folds: {fc}')
  sub_folds = [k_folds[x] for x in fc]
  for fold in sub_folds:
    copy_fold(fold, train_target_dir)
  test_target_dir = fold_dir +'/test'
  print(f'  Test fold: {test_folds[fold_idx]}')
  copy_fold(k_folds[test_folds[fold_idx]], test_target_dir)

Train folds: [(0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 3, 4), (0, 2, 3, 4), (1, 2, 3, 4)]
Test folds: [4, 3, 2, 1, 0]
Fold: 0
  Train folds: (0, 1, 2, 3)
  Test fold: 4
Fold: 1
  Train folds: (0, 1, 2, 4)
  Test fold: 3
Fold: 2
  Train folds: (0, 1, 3, 4)
  Test fold: 2
Fold: 3
  Train folds: (0, 2, 3, 4)
  Test fold: 1
Fold: 4
  Train folds: (1, 2, 3, 4)
  Test fold: 0
CPU times: user 1.07 s, sys: 3.71 s, total: 4.77 s
Wall time: 10.8 s


In [ ]:
!ls /content/kfolds/
!ls /content/kfolds/fold0/
!ls /content/kfolds/fold0/test
!ls /content/kfolds/fold0/train

fold0  fold1  fold2  fold3  fold4
test  train
Electronic  Pop  Rock
Electronic  Pop  Rock


In [ ]:
!find /content/kfolds/fold0/ -type f | wc -l
!find /content/kfolds/fold0/train/ -type f | wc -l
!find /content/kfolds/fold0/test -type f | wc -l

2398
1919
479


In [ ]:
!find /content/kfolds/fold0/train/Pop -type f | wc -l
!find /content/kfolds/fold0/train/Electronic -type f | wc -l
!find /content/kfolds/fold0/train/Rock -type f | wc -l

633
639
647


In [ ]:
!find /content/kfolds/fold0/test/Pop -type f | wc -l
!find /content/kfolds/fold0/test/Electronic -type f | wc -l
!find /content/kfolds/fold0/test/Rock -type f | wc -l

167
160
152


## File and collect peformance statisitics. 4 folds are used for training, test dataset is used for validation.

In [ ]:
%%time
train_generator = get_data_generator('/content/kfolds/fold0/train', True)
test_generator = get_data_generator('/content/kfolds/fold0/test', False)
model = GenreModel()
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=test_generator, epochs=30)
model.save('/content/CNN_30sec_img_5Fold0_m1_epoch30.keras')
!cp /content/CNN_5_Fold0_m1_epoch20.keras ./data/mel_sg/

Found 1919 images belonging to 3 classes.
Found 479 images belonging to 3 classes.


<timed exec>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
15/15 [==============================] - 46s 3s/step - loss: 3.4257 - accuracy: 0.4445 - val_loss: 1.1360 - val_accuracy: 0.3173
Epoch 2/30
15/15 [==============================] - 41s 3s/step - loss: 1.3363 - accuracy: 0.5425 - val_loss: 1.1657 - val_accuracy: 0.3466
Epoch 3/30
15/15 [==============================] - 41s 3s/step - loss: 0.9583 - accuracy: 0.6076 - val_loss: 1.2130 - val_accuracy: 0.3257
Epoch 4/30
15/15 [==============================] - 41s 3s/step - loss: 0.8766 - accuracy: 0.6399 - val_loss: 1.3571 - val_accuracy: 0.3340
Epoch 5/30
15/15 [==============================] - 41s 3s/step - loss: 0.8056 - accuracy: 0.6623 - val_loss: 1.3790 - val_accuracy: 0.3340
Epoch 6/30
15/15 [==============================] - 41s 3s/step - loss: 0.6747 - accuracy: 0.7108 - val_loss: 1.4873 - val_accuracy: 0.3340
Epoch 7/30
15/15 [==============================] - 41s 3s/step - loss: 0.6509 - accuracy: 0.7202 - val_loss: 1.5993 - val_accuracy: 0.3340
Epoch 8/30
15/15 [==

In [ ]:
%%time
train_generator = get_data_generator('/content/kfolds/fold1/train', True)
test_generator = get_data_generator('/content/kfolds/fold1/test', False)
model = GenreModel()
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=test_generator, epochs=30)
#model.save('/content/CNN_30sec_img_5Fold0_m1_epoch30.keras')
#!cp /content/CNN_5_Fold0_m1_epoch20.keras ./data/mel_sg/

Found 1919 images belonging to 3 classes.
Found 479 images belonging to 3 classes.


<timed exec>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
15/15 [==============================] - 46s 3s/step - loss: 2.9096 - accuracy: 0.4403 - val_loss: 1.1536 - val_accuracy: 0.3946
Epoch 2/30
15/15 [==============================] - 41s 3s/step - loss: 1.2850 - accuracy: 0.5664 - val_loss: 1.1328 - val_accuracy: 0.3549
Epoch 3/30
15/15 [==============================] - 41s 3s/step - loss: 1.0218 - accuracy: 0.6159 - val_loss: 1.2500 - val_accuracy: 0.3549
Epoch 4/30
15/15 [==============================] - 42s 3s/step - loss: 0.8187 - accuracy: 0.6509 - val_loss: 1.4572 - val_accuracy: 0.3549
Epoch 5/30
15/15 [==============================] - 42s 3s/step - loss: 0.7372 - accuracy: 0.6806 - val_loss: 1.2986 - val_accuracy: 0.3549
Epoch 6/30
15/15 [==============================] - 42s 3s/step - loss: 0.6955 - accuracy: 0.7103 - val_loss: 1.2765 - val_accuracy: 0.3549
Epoch 7/30
15/15 [==============================] - 42s 3s/step - loss: 0.6235 - accuracy: 0.7337 - val_loss: 1.4804 - val_accuracy: 0.3549
Epoch 8/30
15/15 [==

In [ ]:
%%time
train_generator = get_data_generator('/content/kfolds/fold2/train', True)
test_generator = get_data_generator('/content/kfolds/fold2/test', False)
model = GenreModel()
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=test_generator, epochs=30)

Found 1918 images belonging to 3 classes.
Found 480 images belonging to 3 classes.


<timed exec>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
15/15 [==============================] - 47s 3s/step - loss: 3.9878 - accuracy: 0.4145 - val_loss: 1.2327 - val_accuracy: 0.3396
Epoch 2/30
15/15 [==============================] - 42s 3s/step - loss: 1.6353 - accuracy: 0.5605 - val_loss: 1.2024 - val_accuracy: 0.3396
Epoch 3/30
15/15 [==============================] - 42s 3s/step - loss: 0.9689 - accuracy: 0.6189 - val_loss: 1.2327 - val_accuracy: 0.3042
Epoch 4/30
15/15 [==============================] - 43s 3s/step - loss: 0.8317 - accuracy: 0.6418 - val_loss: 1.4814 - val_accuracy: 0.3042
Epoch 5/30
15/15 [==============================] - 43s 3s/step - loss: 0.7384 - accuracy: 0.6887 - val_loss: 1.5175 - val_accuracy: 0.3042
Epoch 6/30
15/15 [==============================] - 43s 3s/step - loss: 0.6723 - accuracy: 0.7289 - val_loss: 1.7236 - val_accuracy: 0.3042
Epoch 7/30
15/15 [==============================] - 42s 3s/step - loss: 0.6458 - accuracy: 0.7336 - val_loss: 1.6859 - val_accuracy: 0.3042
Epoch 8/30
15/15 [==

In [ ]:
%%time
train_generator = get_data_generator('/content/kfolds/fold3/train', True)
test_generator = get_data_generator('/content/kfolds/fold3/test', False)
model = GenreModel()
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=test_generator, epochs=30)

Found 1918 images belonging to 3 classes.
Found 480 images belonging to 3 classes.


<timed exec>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
15/15 [==============================] - 46s 3s/step - loss: 2.9504 - accuracy: 0.4447 - val_loss: 1.1356 - val_accuracy: 0.3333
Epoch 2/30
15/15 [==============================] - 42s 3s/step - loss: 1.2903 - accuracy: 0.5412 - val_loss: 1.1663 - val_accuracy: 0.4604
Epoch 3/30
15/15 [==============================] - 42s 3s/step - loss: 1.0147 - accuracy: 0.5787 - val_loss: 1.1625 - val_accuracy: 0.3458
Epoch 4/30
15/15 [==============================] - 42s 3s/step - loss: 0.8411 - accuracy: 0.6559 - val_loss: 1.2568 - val_accuracy: 0.3479
Epoch 5/30
15/15 [==============================] - 42s 3s/step - loss: 0.7410 - accuracy: 0.6809 - val_loss: 1.5903 - val_accuracy: 0.3479
Epoch 6/30
15/15 [==============================] - 42s 3s/step - loss: 0.6915 - accuracy: 0.7075 - val_loss: 1.6081 - val_accuracy: 0.3479
Epoch 7/30
15/15 [==============================] - 42s 3s/step - loss: 0.6070 - accuracy: 0.7351 - val_loss: 1.5187 - val_accuracy: 0.3479
Epoch 8/30
15/15 [==

In [ ]:
%%time
train_generator = get_data_generator('/content/kfolds/fold4/train', True)
test_generator = get_data_generator('/content/kfolds/fold4/test', False)
model = GenreModel()
opt = Adam(learning_rate=0.0005)
model.compile(optimizer = opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, validation_data=test_generator, epochs=30)

Found 1918 images belonging to 3 classes.
Found 480 images belonging to 3 classes.


<timed exec>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/30
15/15 [==============================] - 47s 3s/step - loss: 3.2145 - accuracy: 0.4338 - val_loss: 1.1353 - val_accuracy: 0.3542
Epoch 2/30
15/15 [==============================] - 43s 3s/step - loss: 1.3382 - accuracy: 0.5511 - val_loss: 1.1608 - val_accuracy: 0.4479
Epoch 3/30
15/15 [==============================] - 42s 3s/step - loss: 0.9516 - accuracy: 0.6064 - val_loss: 1.2037 - val_accuracy: 0.3333
Epoch 4/30
15/15 [==============================] - 42s 3s/step - loss: 0.8286 - accuracy: 0.6517 - val_loss: 1.4655 - val_accuracy: 0.3250
Epoch 5/30
15/15 [==============================] - 42s 3s/step - loss: 0.7697 - accuracy: 0.6674 - val_loss: 1.6104 - val_accuracy: 0.3250
Epoch 6/30
15/15 [==============================] - 42s 3s/step - loss: 0.7713 - accuracy: 0.6835 - val_loss: 1.7760 - val_accuracy: 0.3250
Epoch 7/30
15/15 [==============================] - 42s 3s/step - loss: 0.6464 - accuracy: 0.7237 - val_loss: 1.8046 - val_accuracy: 0.3250
Epoch 8/30
15/15 [==